# Residential Real Estate vs Supply, Inflation, and Interest Rate #

## Description ##
Our goal is to determine how the value of residential real estate relates to supply, inflation, and interest rate.  Additionally, we will compare real estate returns to the returns of the  S&P 500 and XLRE (Real Estate Select Sector).

## Objectives ##
We intend to identify ideal periods to transact residential real estate based on valuation, inflation, and interest rate.

In [ ]:
# Imports
import pandas as pd
import os
from dotenv import load_dotenv
from fredapi import Fred
import hvplot.pandas

# Load Environment Variables
load_dotenv()

# setup fred api client
FRED_API_KEY = os.getenv('FRED_API_KEY')
fred = Fred(api_key=FRED_API_KEY)

In [ ]:
# Inflation, consumer prices for the United States
inflation_cpi_df = fred.get_series("FPCPITOTLZGUSA")
inflation_cpi_df.hvplot()

In [ ]:
# Federal Funds Effective Rate
effective_rate_df = fred.get_series('DFF')
effective_rate_df.hvplot()

In [ ]:
# Monthly Supply of New Houses in the United States 
monthly_supply_df = fred.get_series('MSACSR')
monthly_supply_df.hvplot()